# MySQL Practice Tutorial 🗄️

Welcome to the comprehensive MySQL Practice Tutorial! This interactive notebook will guide you through:

- 🔌 Database connection and configuration
- 📊 Basic CRUD operations
- 🔍 Advanced queries and joins
- 📈 Data analysis and visualization
- 🧪 Performance optimization
- 🔄 Transactions and stored procedures

Let's start your MySQL journey!

In [2]:
# Import required libraries
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Libraries imported successfully!")
print(f"🏠 Project root: {project_root}")

ModuleNotFoundError: No module named 'pandas'

## 1. Database Connection 🔌

First, let's establish a connection to our MySQL database and verify it's working properly.

In [ ]:
# Test database connection
from config.database import test_connection, create_connection

# Test the connection
if test_connection():
    print("✅ Database connection successful!")
    
    # Get a connection for queries
    conn = create_connection()
    cursor = conn.cursor()
    
    # Show database info
    cursor.execute("SELECT VERSION() as version")
    version = cursor.fetchone()
    print(f"📊 MySQL Version: {version[0]}")
    
    cursor.execute("SELECT DATABASE() as current_db")
    db_name = cursor.fetchone()
    print(f"🗄️ Current Database: {db_name[0]}")
    
    cursor.close()
    conn.close()
else:
    print("❌ Database connection failed!")
    print("Please check your .env file and ensure MySQL is running.")

## 2. Data Exploration 🔍

Let's explore our database structure and load some sample data for analysis.

In [ ]:
# Load data using pandas for analysis
from config.database import create_connection

def query_to_dataframe(query, params=None):
    """Execute a query and return results as a pandas DataFrame."""
    conn = create_connection()
    try:
        df = pd.read_sql(query, conn, params=params)
        return df
    finally:
        conn.close()

# Show tables in database
tables_df = query_to_dataframe("SHOW TABLES")
print("📋 Tables in database:")
display(tables_df)

# Get table information
def show_table_info(table_name):
    print(f"\n📊 Structure of {table_name}:")
    info_df = query_to_dataframe(f"DESCRIBE {table_name}")
    display(info_df)
    
    count_df = query_to_dataframe(f"SELECT COUNT(*) as row_count FROM {table_name}")
    print(f"📈 Row count: {count_df.iloc[0]['row_count']}")

# Show info for each table
for table in tables_df.iloc[:, 0]:
    show_table_info(table)

## 3. Data Visualization 📈

Now let's create some interesting visualizations from our database data.

In [ ]:
# Sales analysis visualization
sales_df = query_to_dataframe("""
    SELECT 
        DATE(o.order_date) as order_date,
        SUM(oi.quantity * oi.price) as daily_revenue,
        COUNT(DISTINCT o.id) as orders_count,
        AVG(oi.quantity * oi.price) as avg_order_value
    FROM orders o
    JOIN order_items oi ON o.id = oi.order_id
    GROUP BY DATE(o.order_date)
    ORDER BY order_date
""")

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 Sales Dashboard', fontsize=16, fontweight='bold')

# Daily revenue trend
axes[0, 0].plot(sales_df['order_date'], sales_df['daily_revenue'], marker='o', linewidth=2)
axes[0, 0].set_title('💰 Daily Revenue Trend')
axes[0, 0].set_ylabel('Revenue ($)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Orders count
axes[0, 1].bar(sales_df['order_date'], sales_df['orders_count'], alpha=0.7)
axes[0, 1].set_title('📦 Daily Orders Count')
axes[0, 1].set_ylabel('Number of Orders')
axes[0, 1].tick_params(axis='x', rotation=45)

# Average order value
axes[1, 0].plot(sales_df['order_date'], sales_df['avg_order_value'], 
                marker='s', color='green', linewidth=2)
axes[1, 0].set_title('💳 Average Order Value')
axes[1, 0].set_ylabel('AOV ($)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Product category analysis
category_df = query_to_dataframe("""
    SELECT 
        p.category,
        SUM(oi.quantity * oi.price) as revenue,
        COUNT(oi.id) as items_sold
    FROM products p
    JOIN order_items oi ON p.id = oi.product_id
    GROUP BY p.category
    ORDER BY revenue DESC
""")

axes[1, 1].pie(category_df['revenue'], labels=category_df['category'], autopct='%1.1f%%')
axes[1, 1].set_title('🏷️ Revenue by Category')

plt.tight_layout()
plt.show()

# Display summary statistics
print("\n📊 Sales Summary:")
print(f"Total Revenue: ${sales_df['daily_revenue'].sum():,.2f}")
print(f"Total Orders: {sales_df['orders_count'].sum():,}")
print(f"Average Daily Revenue: ${sales_df['daily_revenue'].mean():,.2f}")
print(f"Average Order Value: ${sales_df['avg_order_value'].mean():.2f}")

## 4. Advanced SQL Queries 🔍

Let's practice some advanced SQL techniques including joins, subqueries, and window functions.

In [ ]:
# Complex queries for business insights

# 1. Customer analysis with window functions
customer_analysis = query_to_dataframe("""
    SELECT 
        c.first_name,
        c.last_name,
        COUNT(o.id) as total_orders,
        SUM(oi.quantity * oi.price) as total_spent,
        AVG(oi.quantity * oi.price) as avg_order_value,
        ROW_NUMBER() OVER (ORDER BY SUM(oi.quantity * oi.price) DESC) as spending_rank
    FROM customers c
    LEFT JOIN orders o ON c.id = o.customer_id
    LEFT JOIN order_items oi ON o.id = oi.order_id
    GROUP BY c.id, c.first_name, c.last_name
    ORDER BY total_spent DESC
""")

print("🏆 Top Customers by Spending:")
display(customer_analysis.head())

# 2. Product performance analysis
product_performance = query_to_dataframe("""
    SELECT 
        p.product_name,
        p.category,
        p.price,
        COALESCE(SUM(oi.quantity), 0) as units_sold,
        COALESCE(SUM(oi.quantity * oi.price), 0) as revenue,
        COALESCE(AVG(oi.quantity), 0) as avg_quantity_per_order
    FROM products p
    LEFT JOIN order_items oi ON p.id = oi.product_id
    GROUP BY p.id, p.product_name, p.category, p.price
    ORDER BY revenue DESC
""")

print("\n📦 Product Performance:")
display(product_performance)

# 3. Monthly sales trend
monthly_sales = query_to_dataframe("""
    SELECT 
        DATE_FORMAT(o.order_date, '%Y-%m') as month,
        COUNT(DISTINCT o.id) as orders,
        SUM(oi.quantity * oi.price) as revenue,
        AVG(oi.quantity * oi.price) as avg_order_value
    FROM orders o
    JOIN order_items oi ON o.id = oi.order_id
    GROUP BY DATE_FORMAT(o.order_date, '%Y-%m')
    ORDER BY month
""")

print("\n📅 Monthly Sales Trend:")
display(monthly_sales)

## 5. Next Steps & Resources 🚀

Congratulations! You've completed the MySQL practice tutorial. Here's what you can explore next:

### 🎯 **Continue Learning:**
- Run more examples: `python cli.py examples`
- Try exercises: `python cli.py exercises` 
- Explore performance: `python cli.py benchmark`
- Start the API: `python cli.py api`

### 📚 **Advanced Topics:**
- Stored procedures and functions
- Database optimization and indexing
- Transaction management
- Data warehousing concepts

### 🛠️ **Tools Available:**
- **CLI Tool**: `python cli.py` for interactive mode
- **Docker Environment**: Isolated MySQL setup
- **Performance Monitoring**: Real-time database metrics  
- **REST API**: Web interface for database operations

### 📊 **Project Structure:**
```
mysql_practice/
├── examples/          # Learning examples
├── exercises/         # Practice exercises  
├── analytics/         # Business intelligence
├── api/              # REST API
├── monitoring/       # Performance tools
├── notebooks/        # This tutorial!
└── docs/             # Documentation
```

**Keep practicing and happy querying! 🎉**